<img src="https://github.com/Minyall/sc207_materials/blob/master/images/gephi_network.png?raw=true" align="right" width="300">


# SC207 - Session 8
# Social Network Analysis with Gephi - EXTRA

This notebook demonstrates how to create a network of Tweets to entities. This allows us to later project a simplified graph of entity co-occurence in Gephi later.

In [70]:
import pandas as pd
def extract_entity(entity_dict, entity_key):
    return entity_dict[entity_key]


def extract_entities(df, entity_column, entity_key, new_column_name):
     
    entity_data = df.explode(entity_column).copy()
    
    not_empty = ~entity_data[entity_column].isna()
    entity_data = entity_data[not_empty]
    
    entity_data[new_column_name] = entity_data[entity_column].apply(extract_entity, entity_key=entity_key)
    return entity_data

In [92]:
df = pd.read_pickle('example_twitter_data_unpacked.pkl')

In [94]:
subset = ['id','entities.hashtags']

data = df[subset]
data.head()

,id,entities.hashtags
0,1330137025148817408,[]
1,1330137020711235586,[]
2,1330137019679432707,[]
3,1330137016881860610,[]
4,1330137014180737025,[]


In [98]:
edges = extract_entities(df,entity_column='entities.hashtags', entity_key='text',new_column_name='tag')
edges.head()
edges = edges[['id','tag']]
edges = edges.rename(columns={'id': 'source', 'tag' :'target'})

edges['weight'] = 1
edges = edges.groupby(['source','target']).sum().reset_index()
edges['edge_type'] = 'tagged_with'

edges.head()

,source,target
13,1330136987232301058,Johnson
14,1330136978604560386,BullyPatel
68,1330136850980286465,LoanCharge
80,1330136822849134593,BullyingisNEVERok
89,1330136798501220362,saynotobullying


In [105]:
tweets = edges['source'].to_frame('id')
tags = edges['target'].to_frame('id')

tweets['type'] = 'tweet'
tags['type'] = 'tag'
tags['Label'] = tags['id']

nodes = tweets.append(tags).drop_duplicates('id')

nodes.head()

,id,type,Label
13,1330136987232301058,tweet,NaN
14,1330136978604560386,tweet,NaN
68,1330136850980286465,tweet,NaN
80,1330136822849134593,tweet,NaN
89,1330136798501220362,tweet,NaN


In [108]:
nodes.tail()

,id,type,Label,retweet_count,favorite_count
1904,ImmigrationReform,tag,ImmigrationReform,NaN,NaN
1905,ALL,tag,ALL,NaN,NaN
1906,NeverTrustATory,tag,NeverTrustATory,NaN,NaN
1907,FollowBackFriday,tag,FollowBackFriday,NaN,NaN
1908,CORRUPTION,tag,CORRUPTION,NaN,NaN


In [106]:
node_data = df[['id', 'retweet_count','favorite_count']]
nodes = nodes.merge(node_data, left_on='id', right_on='id', how='left')
nodes.head()

,id,type,Label,retweet_count,favorite_count
0,1330136987232301058,tweet,NaN,0.0,0.0
1,1330136978604560386,tweet,NaN,19.0,0.0
2,1330136850980286465,tweet,NaN,8.0,0.0
3,1330136822849134593,tweet,NaN,0.0,0.0
4,1330136798501220362,tweet,NaN,3.0,0.0


In [107]:
nodes.to_csv('tag_node_list.csv',index=False)
edges.to_csv('tag_edge-list.csv',index=False)